### Python code for evaluating English text using the METEOR score

In [2]:
from typing import List
import evaluate # pip install evaluate


def meteor(generation_gt: List[List[str]], generations: List[str],
           alpha: float = 0.9,
           beta: float = 3.0,
           gamma: float = 0.5) -> List[float]:
    """Computes METEOR score."""
    # Note: NLTK resources 'wordnet' and 'punkt' might be needed by evaluate.load("meteor").
    # If you encounter errors, try running the following in a Python interpreter:
    # import nltk
    # nltk.download('wordnet')
    # nltk.download('punkt')
    try:
        meteor_instance = evaluate.load("meteor")
        result = huggingface_evaluate(meteor_instance, 'meteor', generation_gt, generations,
                                      alpha=alpha, beta=beta, gamma=gamma)
        del meteor_instance
        return result
    except Exception as e:
        print(f"Error loading/computing METEOR: {e}")
        return [0.0] * len(generations)


def huggingface_evaluate(instance, key: str,
                         generation_gt: List[List[str]], generations: List[str],
                         **kwargs) -> List[float]:
    """Helper function to compute scores using Hugging Face evaluate."""
    def compute_score(gt: List[str], pred: str) -> float:
        # Computes the score for a single prediction against potentially multiple references. Returns the maximum score if multiple references are provided for one prediction.
        return max(list(map(
            lambda x: instance.compute(predictions=[pred], references=[x], **kwargs)[key], gt)))

    # Map the compute_score function over zipped ground truths and predictions.
    # zip(generation_gt, generations) pairs each list of references with its corresponding prediction.
    result = list(map(lambda x: compute_score(x[0], x[1]), zip(generation_gt, generations)))
    return result

### Example 1 : Individual Sentence Evaluation

In [4]:
if __name__ == "__main__":
    prediction_sentences = [
        "When the E-405 fault is detected on the PCU-2301 controller, technicians should inspect the M39A valve",
        "and ensure the S505 sensor is securely connected to the P-201 pump to resume normal operation."
    ]
    reference_sentences = [
        "If error code E-405 appears on the PCU-2301, check the M39A valve for malfunction and verify that the S505 sensor",
        "is properly linked to the P-291 pump to restore system functionality."
    ]

    print("--- Evaluation ---")
    print("\n1. Individual Sentence Evaluation:")

    generation_gt_individual = [[r] for r in reference_sentences]

    print(f"generation_gt_individual: {generation_gt_individual}")
    print(f"prediction_sentences: {prediction_sentences}")

    meteor_scores_individual = meteor(generation_gt_individual, prediction_sentences)
    for i, score in enumerate(meteor_scores_individual):
        print(f"  Sentence {i+1} METEOR score: {score:.4f}")

--- Evaluation ---

1. Individual Sentence Evaluation:
generation_gt_individual: [['If error code E-405 appears on the PCU-2301, check the M39A valve for malfunction and verify that the S505 sensor'], ['is properly linked to the P-291 pump to restore system functionality.']]
prediction_sentences: ['When the E-405 fault is detected on the PCU-2301 controller, technicians should inspect the M39A valve', 'and ensure the S505 sensor is securely connected to the P-201 pump to resume normal operation.']
  Sentence 1 METEOR score: 0.3334
  Sentence 2 METEOR score: 0.5078


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Example 2 : Full text evaluation (Preferred)

In [5]:

if __name__ == "__main__":
    prediction_sentences = [
        "When the E-405 fault is detected on the PCU-2301 controller, technicians should inspect the M39A valve",
        "and ensure the S505 sensor is securely connected to the P-201 pump to resume normal operation."
    ]
    reference_sentences = [
        "If error code E-405 appears on the PCU-2301, check the M39A valve for malfunction and verify that the S505 sensor",
        "is properly linked to the P-291 pump to restore system functionality."
    ]

    print("--- Evaluation ---")
    print("\n2. Full Text Evaluation:")

    full_prediction = " ".join(prediction_sentences)
    full_reference = " ".join(reference_sentences)


    generation_gt_full = [[full_reference]]
    generations_full = [full_prediction]

    print(f"generation_gt_full: {generation_gt_full}")
    print(f"generations_full: {generations_full}")

    meteor_score_full_list = meteor(generation_gt_full, generations_full)

    if meteor_score_full_list:
        print(f"  Full text METEOR score: {meteor_score_full_list[0]:.4f}")

--- Evaluation ---

2. Full Text Evaluation:
generation_gt_full: [['If error code E-405 appears on the PCU-2301, check the M39A valve for malfunction and verify that the S505 sensor is properly linked to the P-291 pump to restore system functionality.']]
generations_full: ['When the E-405 fault is detected on the PCU-2301 controller, technicians should inspect the M39A valve and ensure the S505 sensor is securely connected to the P-201 pump to resume normal operation.']
  Full text METEOR score: 0.5888


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
